In [ ]:
from dotenv import load_dotenv
import locale
import numpy as np
import os
import pandas as pd
import sqlalchemy as sa

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

engine = sa.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
new_games = [
    {"concurso": 2745, "data": "2024-07-04", "bola1": 2, "bola2": 5, "bola3": 7, "bola4": 11, "bola5": 52, "bola6": 57, "ganhou": True},
]

df_games = pd.DataFrame(new_games)
rows_games = df_games.to_sql(name="megasena", con=engine, if_exists="append", index=False)
print(f"Foram {rows_games} jogo(s) inserido(s) com sucesso!")

In [ ]:
df_last = pd.read_sql(sql="SELECT * FROM megasena", con=engine)

df_last["concurso"] = df_last["concurso"].astype(str).str.zfill(4)
df_last["data"] = pd.to_datetime(df_last["data"]).dt.strftime("%x (%a)")
df_last[df_last.columns[2:-1]] = df_last[df_last.columns[2:-1]].map(lambda x: f"{x:02}")
df_last["ganhou"] = np.where(df_last["ganhou"], "Sim", "Não")
df_last.columns = ["Concurso", "Data", "Bola 1", "Bola 2", "Bola 3", "Bola 4", "Bola 5", "Bola 6", "Ganhou?"]

df_last.tail(15)

In [ ]:
stmt = "SELECT * FROM megasena WHERE data IN (SELECT MAX(data) FROM megasena GROUP BY YEAR(data) HAVING YEAR(data) <> YEAR(CURRENT_DATE))"
df_mega_da_virada = pd.read_sql(stmt, con=engine)

df_mega_da_virada["concurso"] = df_mega_da_virada["concurso"].astype(str).str.zfill(4)
df_mega_da_virada["data"] = pd.to_datetime(df_mega_da_virada["data"]).dt.strftime("%x (%a)")
df_mega_da_virada[df_mega_da_virada.columns[2:-1]] = df_mega_da_virada[df_mega_da_virada.columns[2:-1]].map(lambda x: f"{x:02}")
df_mega_da_virada["ganhou"] = np.where(df_mega_da_virada["ganhou"], "Sim", "Não")
df_mega_da_virada.columns = ["Concurso", "Data", "Bola 1", "Bola 2", "Bola 3", "Bola 4", "Bola 5", "Bola 6", "Ganhou?"]

df_mega_da_virada